In [1]:
#numeric python module, useful for handling large data arrays
import numpy as np

#module for dealing with netCDF files (all sat files are netCDF)
import netCDF4 as nc

#modules for making plots. Here wer are importing the full matplotlib
#module, but also the sub-module pyplot separately with an abbreviated
#name so it is easier to use in the code
import matplotlib.pyplot as plt
import matplotlib

#for displaying the plots in this notebook 
%matplotlib inline

#module for making maps
from mpl_toolkits.basemap import Basemap

#miscellaneous operating system interfaces module for doing things like
#moving file, exploring directory paths, etc..
import os
import re

In [5]:
#specify the directory where all the files are saved
L3filesdir= '/Users/clarabirdferrer/Documents/Bigelow/SatelliteData/Kd490/all_2014-16'

#list all the files in the directory using the os module
L3files_all=os.listdir(L3filesdir)

#combining the directory path with the file names
#to have a list of files with the full path
L3files_all = [L3filesdir + '/' + f for f in L3files_all]

In [6]:

#specifying the edges of the subscene in lat & lon coordinates
#subscene = Gulf of Maine
east = -65
west = -71.5
north = 45.25
south = 42.0


#defining a function that searches for the element
#nearest a particular values within an array, and returns
#the position (or index) of that element
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

#files with .nc and separating files by algorithm product
ncfiles=[]
for ff in L3files_all:
    M = re.search('.nc',ff)
    if M:
        ncfiles += [ff]

In [7]:
###looping through each file based on algorithm and saving in folder based on algorithm           
            
            
##GIOP ALORITHM LOOP            
#number of L3 files
numfiles = len(ncfiles)
        
#looping through each file, doing as detailed in the above 
#section for subsetting data
L3data = {}
for ii in range(0,numfiles):
    L3file = ncfiles[ii]
    fileinfo = nc.Dataset(L3file)
    L3data['lat'] = fileinfo.variables['lat'][:]
    L3data['lon'] = fileinfo.variables['lon'][:]
    L3data['KD490'] = fileinfo.variables['Kd_490'][:,:]
    
    L3data['lon'],L3data['lat']=np.meshgrid(L3data['lon'],L3data['lat'])
    
    #turning the lats and lons into indices within the data array
    #so we can then extract the data easily using the above defined func
    right = find_nearest(L3data['lon'][0,:],east)
    left = find_nearest(L3data['lon'][0,:],west)
    top = find_nearest(L3data['lat'][:,0],north)
    bottom = find_nearest(L3data['lat'][:,0],south)
        
    #extracting the subscene
    subset = L3data['KD490'][top:bottom,left:right]
    subset_lon = L3data['lon'][top:bottom,left:right]
    subset_lat = L3data['lat'][top:bottom,left:right]
    
    #extract date sequence from file name
    r = re.compile("[0-9]{7}")
    m = r.search (L3file)
    if m:
        d = m.group(0)
        month = d[4:]
        year = d[:4]
        
        if month == '001':   ###this step takes all the leap year julian calendar month start dates and makes then non-leap year
            month = '001'
        elif month == '032':
            month = '032'
        elif month == '061':
            month = '060'
        elif month == '092':
            month = '091'
        elif month == '122':
            month = '121'
        elif month == '153':
            month = '152'
        elif month == '183':
            month = '182'
        elif month == '214':
            month = '213'
        elif month == '245':
            month = '244'
        elif month == '275':
            month = '274'
        elif month == '306':
            month = '305'
        elif month == '336':
            month = '335'
            
    
        if month == '001': ###converts julian calendar start dates to typical month numbers --> could also switch out for letter month abbreviations (Jan etc..)
            month = '1'
        elif month == '032':
            month = '2'
        elif month == '060':
            month = '3'
        elif month == '091':
            month = '4'
        elif month == '121':
            month = '5'
        elif month == '152':
            month = '6'
        elif month == '182':
            month = '7'
        elif month == '213':
            month = '8'
        elif month == '244':
            month = '9'
        elif month == '274':
            month = '10'
        elif month == '305':
            month = '11'
        elif month == '335':
            month = '12' 
       
        date = year + '_' + month
        #print(date)
        
    #subset of values to nan
    
    subset[subset < -30000] = np.nan
    
    #save files as the loop runs
    datafile = '/Users/clarabirdferrer/Documents/Bigelow/SatelliteData/Kd490/subsets/kd490_subset_data'
    latfile = '/Users/clarabirdferrer/Documents/Bigelow/SatelliteData/Kd490/subsets/kd490_subset_lat'
    lonfile ='/Users/clarabirdferrer/Documents/Bigelow/SatelliteData/Kd490/subsets/kd490_subset_lon'

    np.savetxt(datafile + '_' + year + '_' + month, subset, fmt='%.6f')
    np.savetxt(latfile + '_' + year + '_' + month, subset_lat, fmt='%.6f')
    np.savetxt(lonfile + '_' + year + '_' + month, subset_lon, fmt='%.6f')

   

/Users/clarabirdferrer/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:96: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
